In [1]:
import numpy as np
import random
import networkx as nx
from matplotlib import pyplot as plt
import pandas as pd
import copy
import os
import sys
import networkx as nx
from scipy.sparse import lil_matrix, coo_matrix
from scipy.io import mmwrite

In [2]:
# moduleー
current_dir = os.path.dirname(os.path.abspath("__file__"))
sys.path.append( str(current_dir) + '/../' )

In [3]:
from setting_param import MakeSample_repeat1_link_prediction_appeared_InputDir as InputDir
from setting_param import MakeSample_repeat1_link_prediction_appeared_utilize_existing_attribute_OutputDir as OutputDir_0
from setting_param import MakeSample_repeat1_link_prediction_appeared_utilize_lost_OutputDir as OutputDir_1
from setting_param import MakeSample_repeat1_link_prediction_appeared_utilize_new_attribute_link_OutputDir as OutputDir_2
from setting_param import MakeSample_repeat1_link_prediction_appeared_utilize_disappeared_OutputDir as OutputDir_4
from setting_param import MakeSample_repeat1_link_prediction_appeared_utilize_appeared_OutputDir as OutputDir_8
OutputDir = {0:OutputDir_0, 1:OutputDir_1, 2:OutputDir_2, 4:OutputDir_4, 8:OutputDir_8}

from setting_param import Evaluation_prediction_num_of_node_new_LSTM_InputDir as predicted_num_InputDir
from setting_param import L
from setting_param import attribute_dim
from setting_param import all_node_num
from setting_param import n_expanded
from setting_param import best_methods

In [4]:
for c_idx, Dir in OutputDir.items():
    os.mkdir(Dir)
    os.mkdir(Dir + "/input/")
    os.mkdir(Dir + "/input/node_attribute/")
    os.mkdir(Dir + "/input/adjacency")
    os.mkdir(Dir + "/label/")
    os.mkdir(Dir + "/mask/")

In [5]:
# READ EXIST_TABLE
EXIST_TABLE = np.load(InputDir + '/exist_table.npy')
EXIST_TABLE.shape

(3859, 68)

In [6]:
n_node = EXIST_TABLE.shape[0]

In [7]:
def ExistNodeList(ts):
    assert ts >= 0, "ts < 0 [referrence error]"
    return np.where(EXIST_TABLE[:, ts]==1)[0]

def GetAppearedNodes(ts):
    return set(ExistNodeList(ts)) - set(ExistNodeList(ts-1))

def GetObservedNodes(ts, L):
    U = set()
    for i in range(L):
        U |= set(ExistNodeList(ts-i))
    return U

def GetNodes(ts, L, node_type):
    if node_type=='all':
        node_set = set(ExistNodeList(ts))
    elif node_type=='stay':
        node_set = set(ExistNodeList(ts-1)) & set(ExistNodeList(ts))
    elif node_type=='lost':
        node_set = set(ExistNodeList(ts-1)) - set(ExistNodeList(ts))
    elif node_type=='return':
        node_set = GetAppearedNodes(ts) - (GetAppearedNodes(ts) - GetObservedNodes(ts-1, L))
    elif node_type=='new':
        node_set = GetAppearedNodes(ts) - GetObservedNodes(ts-1, L)
        node_set |= GetNodes(ts, L, 'return')
    return node_set

In [8]:
def Nx(ts):
    return  nx.from_numpy_matrix(np.load(InputDir + '/adjacency' + str(ts) + '.npy'))

def SubNxNew(ts, L):
    return nx.Graph(Nx(ts).edges(GetNodes(ts, L, 'new')))

def SubNxLost(ts, L):
    return nx.Graph(Nx(ts-1).edges(GetNodes(ts, L, 'lost')))

def GetEdges(ts, L, edge_type):
    G_1 = Nx(ts)
    if edge_type == "all":
        edge_set = G_1.edges
    elif edge_type == 'stay':
        G_0 = Nx(ts - 1)
        edge_set = G_0.edges & G_1.edges
    elif edge_type == "appeared":
        G_0 = Nx(ts - 1)
        edge_set = G_1.edges - G_0.edges - SubNxNew(ts, L).edges
    elif edge_type == "disappeared":
        G_0 = Nx(ts - 1)
        edge_set = G_0.edges - G_1.edges - SubNxLost(ts, L).edges
    return edge_set

In [9]:
def get_adjacency_matrix(ts, L, edge_type):
    G = nx.Graph(list(GetEdges(ts, L, edge_type)))
    A = np.array(nx.to_numpy_matrix(G, nodelist=[i for i in range(n_node)]))
    return A

def get_exist_matrix(ts):
    index = np.where(EXIST_TABLE[:, ts] == 1)[0]
    exist_row = np.zeros((n_node, n_node))
    exist_row[index] = 1
    exist_col = np.zeros((n_node, n_node))
    exist_col[:, index] = 1
    return exist_row * exist_col

def NodeAttribute(ts):
    return  np.load(InputDir + '/node_attribute' + str(ts) + '.npy')

In [10]:
def TsSplit(ts, L):
    ts_train = [(ts+l) for l in range(L)]
    ts_test = ts_train[-1]+1
    ts_all = ts_train.copy()
    ts_all.extend([ts_test])
    return ts_train, ts_test, ts_all

In [11]:
def concat_train_valid_test(train_result, valid_result, test_result):
    result = []
    for train_r in train_result:
        result.append(train_r)
    for valid_r in valid_result:
        result.append(valid_r)
    for test_r in test_result:
        result.append(test_r)
    ts_result_dic = {}
    for t_idx, ts in enumerate(range(L, EXIST_TABLE.shape[1]-L)):
        ts_result_dic[ts] = result[t_idx]
    return ts_result_dic

In [12]:
from repeat_utils.graph_prediction import link_prediction
train_result = link_prediction(best_methods["n_appeared"], best_methods["p_appeared"], best_methods["n_disappeared"], best_methods["p_disappeared"], best_methods["n_new"], best_methods["p_new"], best_methods["n_lost"],  best_methods["p_lost"], True, False, False)
valid_result = link_prediction(best_methods["n_appeared"], best_methods["p_appeared"], best_methods["n_disappeared"], best_methods["p_disappeared"], best_methods["n_new"], best_methods["p_new"], best_methods["n_lost"],  best_methods["p_lost"], False, True, False)
test_result = link_prediction(best_methods["n_appeared"], best_methods["p_appeared"], best_methods["n_disappeared"], best_methods["p_disappeared"], best_methods["n_new"], best_methods["p_new"], best_methods["n_lost"],  best_methods["p_lost"], False, False, True)
pred_adjacency_matrix = concat_train_valid_test(train_result, valid_result, test_result)

/Users/shohei/work/NBA/data/learning_result/prediction_num_of_edge/appeared/LSTM
/Users/shohei/work/NBA/data/learning_result/link_prediction_appeared/GCN
recall:  0.5177057598217314 precision:  0.501242906055063 f1:  0.5093413401963587
/Users/shohei/work/NBA/data/learning_result/prediction_num_of_edge/disappeared/LSTM
/Users/shohei/work/NBA/data/learning_result/link_prediction_disappeared/DynGEM
recall:  0.31060214952832405 precision:  0.308805002099762 f1:  0.3097009686929664
/Users/shohei/work/NBA/data/learning_result/prediction_num_of_edge/new/LSTM
/Users/shohei/work/NBA/data/learning_result/link_prediction_new/COSSIMMLP/PROSER/mix
recall:  0.23621003182300349 precision:  0.20576199590786087 f1:  0.2199372111891072
/Users/shohei/work/NBA/data/learning_result/prediction_num_of_node/lost/LSTM
/Users/shohei/work/NBA/data/learning_result/node_prediction_lost/STGGNN
recall:  0.6466836734693877 precision:  0.6190476190476191 f1:  0.6325639426076107
recall:  0.7053196538756435 precision:  

In [13]:
len(pred_adjacency_matrix)

62

In [14]:
pred_adjacency_matrix[3].shape

(16, 3976, 3976)

In [15]:
# existing nodeattributeEvaluation_inputDir (best_method)
# concatTm [i for i in range(30)], Pos [i for i in range(30, 33)], Age [33], PTS [34]
from setting_param import best_methods_attribute_existing_OutputDir
from repeat_utils.get_predicted_existing_attribute import get_predicted_existing_attribute
best_methods_attribute_existing_OutputDir

{'Tm': '/Users/shohei/work/NBA/data/learning_result/attribute_prediction_exist_Tm/binary_transfer',
 'Pos': '/Users/shohei/work/NBA/data/learning_result/attribute_prediction_exist_Pos/Baseline',
 'Age': '/Users/shohei/work/NBA/data/learning_result/attribute_prediction_exist_Age/STGGNN',
 'PTS': '/Users/shohei/work/NBA/data/learning_result/attribute_prediction_exist_PTS/LSTM'}

In [16]:
train_result = get_predicted_existing_attribute(best_methods_attribute_existing_OutputDir['Tm'], True, False, False)
valid_result = get_predicted_existing_attribute(best_methods_attribute_existing_OutputDir['Tm'], False, True, False)
test_result = get_predicted_existing_attribute(best_methods_attribute_existing_OutputDir['Tm'], False, False, True)
pred_existing_attribute_Tm = concat_train_valid_test(train_result, valid_result, test_result)

In [17]:
train_result = get_predicted_existing_attribute(best_methods_attribute_existing_OutputDir['Pos'], True, False, False)
valid_result = get_predicted_existing_attribute(best_methods_attribute_existing_OutputDir['Pos'], False, True, False)
test_result = get_predicted_existing_attribute(best_methods_attribute_existing_OutputDir['Pos'], False, False, True)
pred_existing_attribute_Pos = concat_train_valid_test(train_result, valid_result, test_result)

In [18]:
train_result = get_predicted_existing_attribute(best_methods_attribute_existing_OutputDir['Age'], True, False, False)
valid_result = get_predicted_existing_attribute(best_methods_attribute_existing_OutputDir['Age'], False, True, False)
test_result = get_predicted_existing_attribute(best_methods_attribute_existing_OutputDir['Age'], False, False, True)
pred_existing_attribute_Age = concat_train_valid_test(train_result, valid_result, test_result)

In [19]:
train_result = get_predicted_existing_attribute(best_methods_attribute_existing_OutputDir['PTS'], True, False, False)
valid_result = get_predicted_existing_attribute(best_methods_attribute_existing_OutputDir['PTS'], False, True, False)
test_result = get_predicted_existing_attribute(best_methods_attribute_existing_OutputDir['PTS'], False, False, True)
pred_existing_attribute_PTS = concat_train_valid_test(train_result, valid_result, test_result)

In [20]:
from setting_param import attribute_prediction_exist_Tm_n_class as Tm_n_class
from setting_param import attribute_prediction_exist_Pos_n_class as Pos_n_class
pred_existing_attribute = {}
for ts in range(L, EXIST_TABLE.shape[1]-L):
    ts_train, ts_test, ts_all = TsSplit(ts, L)
    pred_Tm = np.identity(Tm_n_class)[np.array(pred_existing_attribute_Tm[ts][:, 0], dtype='int64')]
    pred_Pos = np.identity(Pos_n_class)[np.array(pred_existing_attribute_Pos[ts][:, 0], dtype='int64')]
    pred_Age = pred_existing_attribute_Age[ts]
    pred_PTS = pred_existing_attribute_PTS[ts]
    pred_existing_attribute[ts] = np.concatenate([pred_Tm, pred_Pos, pred_Age, pred_PTS], axis=1)
pred_existing_attribute[ts].shape

(3859, 35)

In [21]:
# new nodeattributeEvaluation_inputDir (best_method)
from setting_param import best_methods_attribute_new_OutputDir
from repeat_utils.get_predicted_new_attribute import get_predicted_new_attribute

In [22]:
best_methods_attribute_new_OutputDir

'/Users/shohei/work/NBA/data/learning_result/attribute_prediction_new/PROSER'

In [23]:
train_new, train_teacher, train_new_num, train_teacher_num, train_teacher_idx, train_node_pair_list = get_predicted_new_attribute(best_methods_attribute_new_OutputDir, True, False, False)
valid_new, valid_teacher, valid_new_num, valid_teacher_num, valid_teacher_idx, valid_node_pair_list = get_predicted_new_attribute(best_methods_attribute_new_OutputDir, False, True, False)
test_new, test_teacher, test_new_num, test_teacher_num, test_teacher_idx, test_node_pair_list = get_predicted_new_attribute(best_methods_attribute_new_OutputDir, False, False, True)

In [24]:
pred_new_attribute_new = concat_train_valid_test(train_new, valid_new, test_new)
pred_new_attribute_teacher = concat_train_valid_test(train_teacher, valid_teacher, test_teacher)
pred_new_attribute_new_num = concat_train_valid_test(train_new_num, valid_new_num, test_new_num)
pred_new_attribute_teacher_num = concat_train_valid_test(train_teacher_num, valid_teacher_num, test_teacher_num)
pred_new_attribute_teacher_idx = concat_train_valid_test(train_teacher_idx, valid_teacher_idx, test_teacher_idx)
pred_new_attribute_node_pair_list = concat_train_valid_test(train_node_pair_list, valid_node_pair_list, test_node_pair_list)

In [25]:
ts = 3
print(pred_new_attribute_new[ts].shape)
print(pred_new_attribute_teacher[ts].shape)
print(pred_new_attribute_new_num[ts])
print(pred_new_attribute_teacher_num[ts])
print(pred_new_attribute_teacher_idx[ts])
print(pred_new_attribute_node_pair_list[ts].shape)

(89, 35)
(117, 35)
56
38
[ 28.  45.  55. 169. 221. 232. 233. 285. 286. 287. 288. 289. 290. 291.
 292. 293. 294. 295. 296. 297. 298. 299. 300. 301. 302. 303. 304. 305.
 306. 307. 308. 309. 310. 311. 312. 313. 314. 315.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.]
(56, 2)


In [26]:
# predicted_new_node_num (reference check )
predicted_new_node_num_list = []
for ts in range(L, EXIST_TABLE.shape[1]-L):
    predicted_new_node_num = int(np.load(predicted_num_InputDir + '/output/pred' + str(ts) + '.npy')[0])
    predicted_new_node_num_list.append(predicted_new_node_num)
max_predicted_new_node_num = max(predicted_new_node_num_list)
max_predicted_new_node_num

89

In [27]:
# new_node_num (reference check )
new_node_num_list = []
for ts in range(L, EXIST_TABLE.shape[1]-L):
    ts_train, ts_test, ts_all = TsSplit(ts, L)
    new_node_num = len(GetNodes(ts_test, L, 'new'))
    new_node_num_list.append(new_node_num)
max_new_node_num = max(new_node_num_list)
max_new_node_num

117

In [28]:
# reference check ()
for ts in range(L, EXIST_TABLE.shape[1]-L):
    ts_train, ts_test, ts_all = TsSplit(ts, L)
    
    new, teacher, new_num, teacher_num, teacher_idx, node_pair_list = pred_new_attribute_new[ts], pred_new_attribute_teacher[ts], pred_new_attribute_new_num[ts], pred_new_attribute_teacher_num[ts], pred_new_attribute_teacher_idx[ts], pred_new_attribute_node_pair_list[ts]
    print(new.shape)
    print(teacher.shape)
    print(new_num)
    print(teacher_num)
    print(teacher_idx)
    print(node_pair_list.shape)
    
    # reference check
    assert sorted(GetNodes(ts_test, L, 'new')) == teacher_idx.tolist()[:teacher_num], 'reference error'
    predicted_new_node_num = int(np.load(predicted_num_InputDir + '/output/pred' + str(ts) + '.npy')[0])
    assert new_num == predicted_new_node_num, 'reference error'
    new_node_num = len(GetNodes(ts_test, L, 'new'))
    assert teacher_num == new_node_num, 'reference error'
    assert new.shape[0] == max_predicted_new_node_num, 'reference error'
    assert teacher.shape[0] == max_new_node_num, 'reference error'

(89, 35)
(117, 35)
56
38
[ 28.  45.  55. 169. 221. 232. 233. 285. 286. 287. 288. 289. 290. 291.
 292. 293. 294. 295. 296. 297. 298. 299. 300. 301. 302. 303. 304. 305.
 306. 307. 308. 309. 310. 311. 312. 313. 314. 315.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.]
(56, 2)
(89, 35)
(117, 35)
55
32
[159. 237. 261. 265. 274. 284. 316. 317. 318. 319. 320. 321. 322. 323.
 324. 325. 326. 327. 328. 329. 330. 331. 332. 333. 334. 335. 336. 337.
 338. 339. 340. 341.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   

(89, 35)
(117, 35)
83
83
[1838. 1946. 2000. 2004. 2023. 2041. 2198. 2277. 2291. 2330. 2359. 2421.
 2447. 2475. 2496. 2624. 2625. 2626. 2627. 2628. 2629. 2630. 2631. 2632.
 2633. 2634. 2635. 2636. 2637. 2638. 2639. 2640. 2641. 2642. 2643. 2644.
 2645. 2646. 2647. 2648. 2649. 2650. 2651. 2652. 2653. 2654. 2655. 2656.
 2657. 2658. 2659. 2660. 2661. 2662. 2663. 2664. 2665. 2666. 2667. 2668.
 2669. 2670. 2671. 2672. 2673. 2674. 2675. 2676. 2677. 2678. 2679. 2680.
 2681. 2682. 2683. 2684. 2685. 2686. 2687. 2688. 2689. 2690. 2691.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.]
(83, 2)
(89, 35)
(117, 35)
82
83
[1588. 2089. 2145. 2209. 2247. 2312. 2364. 2394. 2443. 2484. 2522. 2555.
 2585. 2592. 2692. 2693. 2694. 2695. 2696. 2697. 2698. 2699. 2700. 2701.
 2702. 2703. 2704. 2705. 2706. 2707. 2708. 2709. 2710. 2711. 2712. 2713.
 2714. 271

In [29]:
pred_attribute = {}
for c_idx in range(16):
    if not c_idx in [0, 1, 2, 4, 8]:
        # 0 (ー), 1 (lost), 2 (new(＋)), 4 (disappeared), 8 (appeared) 
        continue
    pred_attribute[c_idx] = {}
    for ts in range(L, EXIST_TABLE.shape[1]-L):
        # existing nodeattribute  new nodeattribute  concat
        # existing nodeattribute
        pred_attribute_e = pred_existing_attribute[ts] # (3859, 35)
        # new node attribute
        pred_attribute_n = pred_new_attribute_new[ts] # (89, 35)
        if c_idx == 0:
            pred_attribute[c_idx][ts] = pred_attribute_e
        elif c_idx == 2:
            pred_attribute[c_idx][ts] = np.concatenate([NodeAttribute(ts_train[-1]), pred_attribute_n], axis=0)
        else:
            pred_attribute[c_idx][ts] = NodeAttribute(ts_train[-1])
        #else:
        #    pred_attribute[c_idx][ts] = np.concatenate([pred_attribute_e, pred_attribute_n], axis=0) # (3948, 35) repeat1ーー

In [30]:
for c_idx in range(16):
    if not c_idx in [0, 1, 2, 4, 8]:
        # 0 (ー), 1 (lost), 2 (new(＋)), 4 (disappeared), 8 (appeared) 
        continue
    for ts in range(L, EXIST_TABLE.shape[1]-L):
        ts_train, ts_test, ts_all = TsSplit(ts, L)
        node_attribute = np.zeros((all_node_num + n_expanded, attribute_dim * L))
        npy_adjacency_matrix = np.zeros((all_node_num + n_expanded, (all_node_num + n_expanded) * L))
        
        # ー,  t-L+2  t  t+1 
        for idx, ts_ in enumerate(ts_train[1:]):
            node_attribute[:all_node_num, attribute_dim*idx : attribute_dim*(idx+1)] = NodeAttribute(ts_)
            npy_adjacency_matrix[:all_node_num, (all_node_num + n_expanded)*idx : (all_node_num + n_expanded)*idx + all_node_num] = get_adjacency_matrix(ts_, L, 'all')

        # t+1
        node_attribute[:pred_attribute[c_idx][ts].shape[0], attribute_dim*(L-1):] = pred_attribute[c_idx][ts]
        npy_adjacency_matrix[:, (all_node_num + n_expanded)*(L-1):] = pred_adjacency_matrix[ts][c_idx]
        
        lil_adjacency_matrix = lil_matrix(npy_adjacency_matrix)
        lil_node_attribute = lil_matrix(node_attribute)
        mmwrite(OutputDir[c_idx] + "/input/node_attribute/" + str(ts), lil_node_attribute)
        mmwrite(OutputDir[c_idx] + "/input/adjacency/" + str(ts), lil_adjacency_matrix)
        
        label = np.zeros((all_node_num + n_expanded,  all_node_num + n_expanded))
        label[:all_node_num, :all_node_num] = get_adjacency_matrix(ts_test, L, "appeared")
        mmwrite(OutputDir[c_idx] + "/label/" + str(ts), lil_matrix(label))
        
        exist_matrix = get_exist_matrix(ts_train[-1])
        np.fill_diagonal(exist_matrix, 0)
        mask = np.zeros((all_node_num + n_expanded,  all_node_num + n_expanded))
        mask[:all_node_num, :all_node_num] = exist_matrix - get_adjacency_matrix(ts_train[-1], L, 'all')
        mmwrite(OutputDir[c_idx] + "/mask/" + str(ts), lil_matrix(mask))